# BLE Remote Control

The remote controller is used to drive the robot, but also for configuration. It has:

* an analog joystick (read by the ESP32 ADC)
* 3 quadrature encoded potentiometer that also server as push buttons
* 3 LEDs

The ESP32 reports changes over a BLE UART service and also listens for updates, e.g. of the LEDs, using a simple binary protocol. One button acts as "power switch" together with a power status led (red): when off, the ESP32 is put into deepsleep to preserve power when operated from a battery. The ESP32 also goes into deepsleep if not connected to a BLE central for more than a minute.

Pressing the button in the deepsleep state resumes operation.

The function of the LEDs is as follows:

* RED: esp32 on: advertising or connected to a BLE central
* GREEN: connected to a BLE central
* BLUE: *spare*

## Protocol

Messages consist of a one byte (character) code, followed by a binary float (4 bytes).

### MCU send

| code | description |
| ---- | ----------- |
| x,y  | joy-stick position |
|  q   | quit button pressed, ESP32 going into deepsleep |
| b,c  | button b,c pressed count (since powerup) |
| 1,2,3| encoder counts |
|  v   | battery voltage |
|  h   | heartbeat count |

   
### MCU receive

| code | description |
| ---- | ----------- |
| R,G,B| led on/off (1/0); only B (blue) is available |
|  Q   | put ESP32 into deepsleep: |
|      | press power button on remote to wake up |

## Configure ESP32

*Note:* when updating code, be sure the ESP32 is not in deepsleep!

In [1]:
%connect robot-esp32
%rsync
%rlist
%softreset

Connected to robot-esp32 @ serial:///dev/ttyUSB0
UPDATE  /lib/remote/joy_axis.py
     67  Jan 20 15:16 2022  boot.py
                            lib/
   2713  Jan 20 15:16 2022      ble_advertising.py
   1147  Jan 20 15:16 2022      filter.py
   4847  Jan 20 16:04 2022      loop.py
                                remote/
    120  Jan 20 15:19 2022          __init__.py
   2525  Jan 20 15:28 2022          ble_uart.py
    512  Jan 20 15:16 2022          button.py
    922  Jan 20 15:16 2022          encoder.py
    909  Jan 21 09:16 2022          joy_axis.py
    137  Jan 20 16:04 2022  main.py
    317  Jan 20 15:16 2022  secrets.py

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
!!!!!   softreset ...     !!!!!
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!


## Functionality

`loop.py`, called on every boot from `main.py` continuously scans the peripherals (using a few helper classes) and BLE messages.

In [4]:
!cat $IOT_PROJECTS/robot/code/esp32/lib/loop.py

from remote import BLE_UART, JoyAxis, Button, Encoder
from machine import ADC, Pin, deepsleep, reset_cause, PWRON_RESET
from struct import pack, unpack
from time import ticks_ms, ticks_diff, sleep_ms
import esp32

"""
Messages:
--------

Format:
pack('>Bf', ord(code), value)

Codes:

a) MCU -> Pi
   x, y: joystick position
   q, b, c: button pressed counts
         Note: button 1 assigned to 'q' (quit = deepsleep)
   1, 2, 2: encoder counts
   v: v_bat [V]
   h: heartbeat count

b) Pi -> MCU
   R, G, B: led on/off (1/0)
   Q: power down remote (deepsleep)
"""

HARTBEAT_MS =  1000     # frequency with which hartbeats are sent [ms]
SHUTDOWN_MS = 60000     # shut down if no connection in specified time [ms]
VBATT_MS    = 60000     # rate at which battery level is sent [ms]

def loop():

    # deepsleep wakeup
    deepsleep_wakeup_pin = Pin(4, mode=Pin.IN, pull=Pin.PULL_UP)
    esp32.wake_on_ext0(deepsleep_wakeup_pin, esp32.WAKEUP_ALL_LOW)

    # BLE
    def rx_cb(data):
        nonlocal r

## Raspberry PI

In [3]:
%%host

import nest_asyncio, sys, os
nest_asyncio.apply()
sys.path.append(os.path.join(os.getenv('IOT_PROJECTS'), 'robot/code/rpi'))

import asyncio
from robot import Remote

class TestRemote(Remote):
    
    async def handle(self, dt: float, code: str, value: float):
        # ignore heartbeat
        if code == 'h': return
        print(f"RECV {code} = {value:8.3f}   dt = {1000*dt:12.4} [ms]")

async with TestRemote() as remote:
    blue = 1
    while not remote.stop:
        await remote.rgb(b=blue)
        blue = 1-blue
        await asyncio.sleep(0.5)

scanning for iot49-robot
connecting to iot49-robot ... connected
RECV v =    4.265   dt =        99.73 [ms]
RECV 2 =   -1.000   dt =        588.0 [ms]
RECV 2 =   -2.000   dt =        2.224 [ms]
RECV 2 =   -3.000   dt =        43.67 [ms]
RECV c =    1.000   dt =        830.6 [ms]
RECV x =    0.053   dt =        780.7 [ms]
RECV x =    0.520   dt =        49.18 [ms]
RECV x =    0.876   dt =        46.89 [ms]
RECV x =    0.675   dt =        53.73 [ms]
RECV y =    0.051   dt =        2.286 [ms]
RECV x =    0.135   dt =         94.3 [ms]
RECV y =   -0.103   dt =        977.4 [ms]
RECV y =   -0.501   dt =        2.145 [ms]
RECV y =   -0.583   dt =        142.4 [ms]
RECV y =   -0.312   dt =        1.174 [ms]
---------- Remote powered down - QUIT!
